# Entrenamiento de BERT con Masked Language Modelling (MLM) - CursoGenAI


En este notebook, utilizaremos la librería `transformers` de Hugging Face para cargar un modelo BERT sin entrenar y aplicaremos el Masked Language Modelling (MLM) utilizando el dataset de literatura de Cervantes. Nuestro objetivo será entrenar el modelo para que pueda predecir palabras enmascaradas en el texto.

### Pasos a seguir:
1. Cargar un modelo de BERT sin entrenar.
2. Cargar los datos del dataset de El Quijote.
3. Aplicar la técnica de Masked Language Modelling (MLM).
4. Guardar el modelo entrenado para su posterior uso.
    

In [16]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import pandas as pd
import torch
from transformers import DataCollatorForLanguageModeling, BertTokenizer, AutoConfig, AutoModelForMaskedLM
from datasets import Dataset
from torch.utils.data import DataLoader
import torch
from transformers import Trainer, TrainingArguments
from torch.quantization import quantize_dynamic

## Cargar un Modelo de BERT sin Entrenar

In [ ]:
# Cargar el modelo BERT para Masked Language Modeling
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForMaskedLM.from_pretrained('bert-base-uncased')

# Verificar que el modelo y el tokenizer se hayan cargado correctamente
print("Modelo y tokenizer de BERT para MLM se han cargado correctamente.")

In [ ]:
# Leer el contenido de un archivo de texto plano
file_path = 'archivos/quijote.txt'

# Cargar el archivo de texto y leer sus líneas
with open(file_path, 'r', encoding='utf-8') as file:
    text_lines = file.readlines()

# Convertir el contenido del archivo en un formato adecuado para el entrenamiento
text_data = [{'text': line.strip()} for line in text_lines if line.strip()]


text_df = pd.DataFrame(text_data)
print("Ejemplo de una instancia del archivo de texto cargado:")
print(text_df.head())

In [ ]:
text_data[0]

## Aplicar Masked Language Modelling (MLM)

In [ ]:
def generate_dataset_with_tokenizer(tokenizer,type_dataset="train"):
    # Convertir el array de datos a un objeto Dataset de Hugging Face
    if type_dataset=="train":
        dataset = Dataset.from_list(text_data[100:])
    else:
        dataset = Dataset.from_list(text_data[:100])

    # Tokenizar el texto del dataset
    def tokenize_function(examples):
        # Tokenizar el texto usando el tokenizer de BERT
        return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

    # Tokenizar el dataset con el método map
    tokenized_dataset = dataset.map(tokenize_function, batched=True)

    # Configurar el DataCollator para Masked Language Modeling (MLM) que enmascara aleatoriamente tokens
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

    return tokenized_dataset, data_collator

tokenized_dataset_train, data_collator = generate_dataset_with_tokenizer(bert_tokenizer, type_dataset="train")
tokenized_dataset_test, _= generate_dataset_with_tokenizer(bert_tokenizer, type_dataset="test")

## Entrenamiento del Modelo BERT

In [ ]:
# Verificar si hay una GPU disponible y configurar el dispositivo
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Dispositivo utilizado para el entrenamiento: {device}")

#model = quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)
# Mover el modelo a la GPU si está disponible
#model.to(device)

# Configurar los argumentos para el entrenamiento
training_args = TrainingArguments(
    output_dir="./CursoGenAI_UAL_04_BERT_MLM_Model_example",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=10,
    report_to='none'  # Evitar reportes innecesarios si solo estamos probando
)

# Configurar el Trainer para el entrenamiento
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test
)

# Entrenar el modelo
trainer.train()

# Ejercicios

## Carga del archivo de evaluación

In [ ]:
# Leer el contenido de un archivo de texto plano
file_path = 'archivos/tormes.txt'

# Cargar el archivo de texto y leer sus líneas
with open(file_path, 'r', encoding='utf-8') as file:
    text_lines = file.readlines()

# Convertir el contenido del archivo en un formato adecuado para el entrenamiento
text_data_tormes = [{'text': line.strip()} for line in text_lines if line.strip()]


text_df = pd.DataFrame(text_data_tormes)
print("Ejemplo de una instancia del archivo de texto cargado:")
print(text_df.head())

def generate_dataset_with_tokenizer(tokenizer,data, type_dataset="train"):
    # Convertir el array de datos a un objeto Dataset de Hugging Face
    if type_dataset=="train":
        dataset = Dataset.from_list(text_data[100:])
    else:
        dataset = Dataset.from_list(text_data[:100])

    # Tokenizar el texto del dataset
    def tokenize_function(examples):
        # Tokenizar el texto usando el tokenizer de BERT
        return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

    # Tokenizar el dataset con el método map
    tokenized_dataset = dataset.map(tokenize_function, batched=True)

    # Configurar el DataCollator para Masked Language Modeling (MLM) que enmascara aleatoriamente tokens
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

    return tokenized_dataset, data_collator

tokenized_dataset_train_tormes, data_collator_tormes = generate_dataset_with_tokenizer(bert_tokenizer,text_data_tormes, type_dataset="train")
tokenized_dataset_test_tormes, _= generate_dataset_with_tokenizer(bert_tokenizer,text_data_tormes, type_dataset="test")

### Ejercicio 1: 
Entrenar BERT desde cero

Guárdalo en el directorio: 'CursoGenAI_UAL_04_BERT_MLM_Model_scratch'

In [ ]:
# Configurar los argumentos para el entrenamiento
training_args = TrainingArguments(
    output_dir="./CursoGenAI_UAL_04_BERT_MLM_Model_scratch",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=10,
    report_to='none'  # Evitar reportes innecesarios si solo estamos probando
)

# Configurar el Trainer para el entrenamiento
trainer_scratch = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator_tormes,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test_tormes
)

# Entrenar el modelo
trainer.train()

### Ejercicio 2:

Hacer un fine-tuning de BERT pre-entrenado.

¿Qué parámetros del bucle de entrenamiento hay que modificar para que se haga un fine-tuning?

Principalmente, el parámetro *learning rate*

In [ ]:
bert_model_finetune = AutoModelForMaskedLM.from_pretrained('bert-base-uncased')

print(bert_model_finetune)
# Congelar todas las capas excepto las últimas (capas más específicas)
for name, param in model.named_parameters():
    # Congelar las capas iniciales (puedes ajustar el rango según lo que quieras congelar)
    if 'encoder.layer' in name and int(name.split('.')[3]) < 8:  # Congelar las primeras 8 capas
        param.requires_grad = False

# Imprimir el estado de los parámetros para verificar
for name, param in model.named_parameters():
    print(f'{name}: {"No entrenable" if not param.requires_grad else "Entrenable"}')  

In [ ]:
# Configurar los argumentos para el fine-tuning
training_args_finetune = TrainingArguments(
    output_dir="./CursoGenAI_UAL_04_BERT_MLM_Model_fine_tuning",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=10,
    report_to='none',
    learning_rate=5e-5  # Cambia el valor del learning rate aquí 
)

# Configurar el Trainer para el fine-tuning
trainer_finetune = Trainer(
    model=bert_model_finetune,
    args=training_args_finetune,
    data_collator=data_collator,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test_tormes
)

# Fine-tuning del modelo pre-entrenado
trainer_finetune.train()

### Ejercicio 3: Evaluación de configuraciones

Vas a evaluar los tres métodos en un archivo de texto dado. ¿Cuál es el que mejor funciona?

In [ ]:
from transformers import BertForMaskedLM, BertTokenizer
# Evaluar la exactitud para cada modelo (entrenado desde cero, fine-tuning y preentrenado)
bert_model_pretrained = BertForMaskedLM.from_pretrained('bert-base-uncased', output_hidden_states=True)
bert_tokenizer_pretrained = BertTokenizer.from_pretrained('bert-base-uncased')

eval_results = trainer.evaluate()

print(f"Exactitud del modelo pre-entrenado: {eval_results}")

In [ ]:
# Evaluar la exactitud para cada modelo (entrenado desde cero, fine-tuning y preentrenado)
bert_model_pretrained = AutoModelForMaskedLM.from_pretrained('CursoGenAI_UAL_04_BERT_MLM_Model_scratch')
bert_tokenizer_pretrained = AutoTokenizer.from_pretrained('bert-base-uncased')

eval_results = trainer_scratch.evaluate()

print(f"Exactitud del modelo entrenado desde el principio: {eval_results:.2f}")

In [ ]:
# Evaluar la exactitud para cada modelo (entrenado desde cero, fine-tuning y preentrenado)
bert_model_pretrained = AutoModelForMaskedLM.from_pretrained('CursoGenAI_UAL_04_BERT_MLM_Model_fine_tuning')
bert_tokenizer_pretrained = AutoTokenizer.from_pretrained('bert-base-uncased')

eval_results = trainer_finetune.evaluate()

print(f"Exactitud del modelo tas haber hecho fine-tuning: {eval_results:.2f}")